In [9]:
import re
import pandas as pd
from pymongo import MongoClient # I'm importing my labrary
client = MongoClient("localhost:27017") # I'm giving my local adress
db = client["Ironhack"] # name of the database
collection = db.get_collection("companies") # name of the collection

In [10]:
collection.find_one() 

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [11]:
# Retrieve all distinct keys in the collection
distinct_keys = set()

for document in collection.find():
    distinct_keys.update(document.keys())

# Filter keys that relate to gaming
gaming_keys = []

for key in distinct_keys:
    gaming_documents = collection.count_documents({key: "gaming"})
    if gaming_documents > 0:
        gaming_keys.append(key)

In [12]:
filter = {"category_code": re.compile(r'\bga\w*')}
projection = {"_id": 0, "category_code": 1}
results = list(collection.find(filter, projection))
categories = {results['category_code'] for results in results}
categories

# Now I know the category name of Companies related to gamming, now i can find a company

{'games_video'}

In [13]:
# Now we can filter for our parameters, our company has 


filter = {"category_code": "games_video", "number_of_employees" : {'$gte': 50, '$lt': 2000}}
projection = {"_id": 0, "name": 1, "number_of_employees":1, "offices.city":1}
results = list(collection.find(filter, projection))
df = pd.DataFrame(results)

In [14]:
normalized_offices = pd.json_normalize(df['offices'])
suffix = '_offices'
normalized_offices.columns = [f'{col}{suffix}' for col in normalized_offices.columns]
df = pd.concat([df, normalized_offices], axis=1)

In [15]:
header_mapping = {
    'name': 'name',
    'number_of_employees': 'employees',    
    'offices': 'offices',
    '0_offices' : 'office1',
    '1_offices' : 'office2',
    '2_offices' : 'office3',
    '3_offices' : 'office4',
    '4_offices' : 'office5',
}
df = df.rename(columns=header_mapping)
df

,name,employees,offices,office1,office2,office3,office4,office5
0,Livestream,120,[{'city': 'New York'}],{'city': 'New York'},None,None,None,None
1,Ustream,250,"[{'city': 'San Francisco'}, {'city': 'Los Ange...",{'city': 'San Francisco'},{'city': 'Los Angeles'},{'city': 'Budapest'},None,None
2,RockYou,106,[{'city': 'Redwood City'}],{'city': 'Redwood City'},None,None,None,None
3,hi5,100,[{'city': 'San Francisco'}],{'city': 'San Francisco'},None,None,None,None
4,Curse,58,"[{'city': 'San Francisco'}, {'city': 'Huntsvil...",{'city': 'San Francisco'},{'city': 'Huntsville'},None,None,None
...,...,...,...,...,...,...,...,...
72,2 Minutes,105,[{'city': 'Paris'}],{'city': 'Paris'},None,None,None,None
73,888 Holdings,931,[],None,None,None,None,None
74,Bigpoint,500,"[{'city': 'Hamburg'}, {'city': 'San Francisco'...",{'city': 'Hamburg'},{'city': 'San Francisco'},{'city': 'Berlin'},{'city': 'Ta 'Xbiex'},None
75,Exent,100,"[{'city': 'New York'}, {'city': 'Petach-Tikva'...",{'city': 'New York'},{'city': 'Petach-Tikva'},{'city': 'San Francisco'},None,None


In [21]:
df.to_csv("../data/df.csv")

In [48]:
df2 = pd.read_csv("../data/df.csv")

In [49]:
df2.drop('offices', axis=1, inplace=True)
df2.fillna(0, inplace=True)
df2 = df2.replace({'\'city\': ': ''}, regex=True)
df2 = df2.replace({'\'}': ''}, regex=True)
df2 = df2.replace({'^\{\'': ''}, regex=True)
df2 = df2.replace({'^{\"': ''}, regex=True)
df2 = df2.replace({'\"}$': ''}, regex=True)

In [56]:
df2.columns

Index(['Unnamed: 0', 'name', 'employees', 'office1', 'office2', 'office3',
       'office4', 'office5'],
      dtype='object')

In [59]:
# Suponiendo que 'df' es tu DataFrame y 'mi_columna' es la columna en la que deseas contar los valores únicos
conteo_valores_unicos = df['office1'].apply(str).nunique()

# Imprimir el número de valores únicos
print(conteo_valores_unicos)


46


In [62]:
# Contar los valores únicos en la columna 'city' y crear un nuevo DataFrame con los recuentos
counts = df['office1'].value_counts().reset_index()

# Renombrar las columnas del nuevo DataFrame
counts.columns = ['city', 'repetition']

# Imprimir el nuevo DataFrame con el recuento de valores únicos en 'city'
counts


,city,repetition
0,{'city': 'San Francisco'},10
1,{'city': 'New York'},7
2,0,7
3,{'city': 'Sunnyvale'},3
4,{'city': 'Los Angeles'},3
5,{'city': 'Shanghai'},3
6,{'city': ''},2
7,{'city': 'London'},2
8,{'city': 'Cologne'},2
9,{'city': 'Paris'},2
